In [1]:
from nltk.collocations import * #Bad form
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import json
import pandas as pd
from pprint import pprint
import re
LOCAL_DATA_PATH = '/Users/willsankey/political_history/data'

In [ ]:
#Notes:
Thinking about the prospect of using the most common words in the debates for feature extractions, see here:
    http://www.nltk.org/book/ch06.html
        
I need to turn these debate paragraphs into features and am still thinking through it.

In [2]:
DATA_LIST = [
            "['102317'].json"#,
#             "['102344'].json",
#             "['119038'].json",
#             "['72776'].json",
#             "['84482'].json",
#             "['102322'].json",
#             "['118971'].json",
#             "['63163'].json",
#             "['78691'].json",
#             "['84526'].json",
#             "['102343'].json",
#             "['119012'].json",
#             "['72770'].json",
#             "['84382'].json"
            ]

In [3]:
REPUBLICANS = [
    'PRESIDENT BUSH',
    'MCCAIN',
    'RYAN',
    'PALIN',
    'GOV ROMNEY',
    'PENCE',
    'TRUMP'
    
]

In [4]:
DEMOCRATS = [
    'SENATOR KERRY',
    'THE PRESIDENT',
    'VICE PRESIDENT JOSEPH BIDEN',
    'OBAMA',
    'BIDEN',
    'KAINE',
    'CLINTON'
]

In [5]:
## Getting the full dataframe

In [6]:
#### Initialize lists to collect data
other_speakers = []
data_list = []

#For each debate in the data...
for debate in DATA_LIST:
    
    with open(LOCAL_DATA_PATH + "/" + debate) as data_file:
        #Load the debate
        data = json.load(data_file)
        for _, paragraph in data.iteritems():
            ##CLASSIFYING SPEAKER INTO DEMOCRAT, REPUBLICAN
            paragraph['speaker'] = re.sub('[!@#$.:]', '', paragraph['speaker']).upper()
            if paragraph['speaker'] in REPUBLICANS:
                paragraph['party'] = 'Republican'
            elif paragraph['speaker'] in DEMOCRATS:
                paragraph['party'] = 'Democrat'
            else:
                other_speakers.append(paragraph['speaker'])
                paragraph['party'] = 'NA'
                
            ##TRIGRAM FEATURES
            tokens = RegexpTokenizer(r'\w+')
            token = tokens.tokenize(paragraph['text'])
            stops = set(stopwords.words('english'))
            filtered_words = [word for word in token if word not in stops]
            #print filtered_words

            bifinder = BigramCollocationFinder.from_words(filtered_words, window_size = 2)
            #trifinder = TrigramCollocationFinder.from_words(filtered_words, window_size = 3)

            bifinder.apply_freq_filter(2)
            #for Bigram use this syntax --> bigram_measures = nltk.collocations.BigramAssocMeasures()
            #for Trigram use this syntax --> trigram_measures = nltk.collocations.TrigramAssocMeasures()
            trigram_measures = nltk.collocations.TrigramAssocMeasures()
            bigram_measures = nltk.collocations.BigramAssocMeasures()
            for key, val in bifinder.ngram_fd.items():
                print key, val
            
            ##APPEND DATA TO LIST
            data_list.append(paragraph)

(u'great', u'schools') 2
(u'great', u'teachers') 2
(u'I', u'believe') 2
(u'The', u'role') 2
(u'role', u'Government') 2
(u'Government', u'capacity') 2
(u'opportunity', u'create') 2
(u'Let', u'start') 2
(u'endowed', u'Creator') 2
(u'I', u'like') 2
(u'health', u'care') 2
(u'traditional', u'Medicare') 2
(u'voucher', u'program') 2
(u'Dodd', u'Frank') 2
(u'clear', u'difference') 2
(u'insurance', u'afford') 2
(u'dropping', u'insurance') 2
(u'2', u'500') 2
(u'I', u'going') 3
(u'going', u'stop') 2
(u'PBS', u'I') 2
(u'I', u'think') 3
(u'issue', u'I') 3
(u'raising', u'taxes') 2
(u'income', u'Americans') 3
(u'middle', u'income') 2
(u'income', u'people') 2
(u'tax', u'cut') 2
(u'I', u'want') 2
(u'burden', u'paid') 2
(u'income', u'Americans') 2
(u'adds', u'deficit') 2
(u'cut', u'adds') 2
(u'reduce', u'burden') 2
(u'tax', u'cut') 3
(u'Government', u'land') 2
(u'health', u'care') 3
(u'role', u'Government') 2
(u'He', u'said') 2
(u'broaden', u'base') 2
(u'said', u'going') 2
(u'Ronald', u'Reagan') 2
(u'2'

In [ ]:
#Data check. Checking that we've captured all presidents in speeches.
#print 'Other speakers that were not classified:', set(other_speakers)

In [60]:
all_data = pd.DataFrame(data_list)

In [69]:
df = all_data[all_data['party'] != 'NA']
del all_data #Cleanup

In [70]:
df.speaker.unique()

array([u'THE PRESIDENT', u'GOV ROMNEY', u'CLINTON', u'TRUMP', u'MCCAIN',
       u'OBAMA', u'RYAN', u'BIDEN', u'VICE PRESIDENT JOSEPH BIDEN',
       u'PENCE', u'KAINE', u'PALIN'], dtype=object)

In [71]:
df.describe()

,date,party,speaker,text,title
count,3899,3899,3899,3899,3899
unique,11,2,12,3844,10
top,"October 4, 2016",Democrat,OBAMA,[,Presidential Candidates Debates: Presidential ...
freq,541,1968,587,14,809


In [72]:
df

,date,party,speaker,text,title
0,"October 3, 2012",Democrat,THE PRESIDENT,"I do, because I think that that is the kind of...","Barack Obama: Presidential Debate in Denver, C..."
2,"October 3, 2012",Republican,GOV ROMNEY,"Well, first, I love great schools. Massachuse...","Barack Obama: Presidential Debate in Denver, C..."
3,"October 3, 2012",Republican,GOV ROMNEY,The role of Government: Look behind us—the Con...,"Barack Obama: Presidential Debate in Denver, C..."
4,"October 3, 2012",Democrat,THE PRESIDENT,But I also believe that Government has the cap...,"Barack Obama: Presidential Debate in Denver, C..."
5,"October 3, 2012",Democrat,THE PRESIDENT,"So in the middle of the Civil War, Abraham Lin...","Barack Obama: Presidential Debate in Denver, C..."
6,"October 3, 2012",Democrat,THE PRESIDENT,And so what I've tried to do as President is t...,"Barack Obama: Presidential Debate in Denver, C..."
7,"October 3, 2012",Democrat,THE PRESIDENT,But what I've also said is let's hire another ...,"Barack Obama: Presidential Debate in Denver, C..."
8,"October 3, 2012",Republican,GOV ROMNEY,"Second, in that line that says, ""We are endowe...","Barack Obama: Presidential Debate in Denver, C..."
9,"October 3, 2012",Republican,GOV ROMNEY,But we also believe in maintaining for individ...,"Barack Obama: Presidential Debate in Denver, C..."
10,"October 3, 2012",Republican,GOV ROMNEY,"I'll elaborate, exactly right. First of all, ...","Barack Obama: Presidential Debate in Denver, C..."


In [ ]:
##With the full dataset now I'll begin analysis

In [78]:
min_set = df[['text', 'party']]

In [79]:
#dict_from_data = min_set.to_dict()

In [100]:
featuresets = [(dict(feature.encode('ascii', 'ignore')), str(label)) for index, (feature, label) in min_set.iterrows()]
#See http://stackoverflow.com/questions/29337714/how-to-run-naive-bayes-from-nltk-with-python-pandas for reference
#And here for ascii fix: http://stackoverflow.com/questions/2365411/python-convert-unicode-to-ascii-without-errors

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [98]:
print len(featuresets)
from random import shuffle
shuffle(featuresets)
print featuresets

3899
[(" I just described what my plan is. And I'm happy to talk to you, Joe, too, if you're out there. Here's your fine -- zero. You won't pay a fine, because...", 'Democrat'), ("We're making progress. Our military is assisting in Iraq. And we're hoping that within the year we'll be able to push ISIS out of Iraq and then, you know, really squeeze them in Syria.", 'Democrat'), ("So you say to yourself, why didn't they make the right deal? This is one of the worst deals ever made by any country in history. The deal with Iran will lead to nuclear problems. All they have to do is sit back 10 years, and they don't have to do much.", 'Republican'), ('You know who voted for it? You might never know. That one. You know who voted against it? Me. I have fought time after time against these pork barrel -- these bills that come to the floor and they have all kinds of goodies and all kinds of things in them for everybody and they buy off the votes. ', 'Republican'), (" ... of what we heard Donald 

In [99]:
from nltk.classify import NaiveBayesClassifier as nbc
train_set, test_set = featuresets[:3000], featuresets[3000:]
# Train a classifier
classifier = nbc.train(train_set)
# Test classifier on "Neo"
print classifier.classify(ethnic_features('Silva'))

AttributeError: 'str' object has no attribute 'items'